# Easy Deploying
_________
##### Packages Requirement

In [1]:
# for mock up of Data Ingestion from Postgre Database
import pandas.io.sql as psql
import psycopg2
import sqlite3
import os

# for reading Data
import pandas as pd

# fro training model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# for Data Preprocessing
import re 

##### Stage 1: Data Ingestion

In [9]:
# Point to Postgre Database and Assign as a variable
con = psycopg2.connect(
            host = "192.168.22.131",
            database="Z1_Cloud_Backup",
            user = "postgres",
            password = "Z1@$2020")
cur = con.cursor()

# Define sql Statement and Execute and Assign as a var named data
cur.execute('''-- from database: Z1_Cloud_Backup
select 
----------------------------
row_number() over(order by last_sync_modify) - 1 as index, id as property_id,
created_at date, concat(concat(latitude, ', '), longitude) as latlong, 
case when record_type is null then 'Others' else record_type end as record_type,
case when type is null then 'Others' else type end as type, 
case when current_use is null then 'Others' when current_use = 'N/A' then 'Others' 
    when current_use = 'null' then 'Others' else current_use end as current_use, 
case when land_shape_type is null then 'Others' 
    when land_shape_type = 'null' then 'Others' else land_shape_type end as  land_shape_type,   
case when site_position is null then 'Others' else site_position end as site_position, 
-- need one more field that represents the prices of properties
----------------------------
-- if width is less than or equal to 6 >>>  'Small Width'
-- if width is greater than 6 until 12 >>> 'Medium'
-- if else >>> 'Large Width'
case when building_width <= 6 then 'Small Width' 
	when building_width <= 12 then 'Medium Width' else 'Large Width' end as width_ranges,
----------------------------
-- if length is less than or equal to 11 >>>  'Small Length'
-- if length is greater than 6 until 12 >>> 'Medium Length'
-- if else >>> 'Large Length'
case when building_length <= 11 then 'Small Length' 
	when building_length <= 16 then 'Medium Length' else 'Large Length' end as length_ranges,
----------------------------
-- listing Fields
is_rent, -- whether the properties are currently opening for rent or not.
is_sale, -- whether the properties are currently opening for sale or not.
is_appraisal, -- whether the properties are already evaluated price or not yet.
----------------------------
-- seperated fields from full_address
split_part(full_address, ',', 5) as address_1, -- could be either provinces or dictricts
split_part(full_address, ',', 4) as address_2, -- could be either districts or communes
split_part(full_address, ',', 3) as address_3  -- could be either communes or villages

from public.properties where created_at > '2022-04-20';''')
data = cur.fetchall()

# to make the data as a dataframe 
cols = []
for elt in cur.description:
    cols.append(elt[0])
pd.set_option("display.max_columns", None)
properties = pd.DataFrame(data = data, columns = cols)

##### Stage 2: Preprocessing and Training Model

In [62]:
def PreprocessingAndTrainingModel():
    # fill categorical fields by "Others" in missing values
    properties[["record_type","type","current_use","land_shape_type","site_position"]] = properties[["record_type","type","current_use","land_shape_type","site_position"]].fillna('Others')
    # fill 0 in missing values of boolean fields
    properties[["is_rent","is_sale","is_appraisal"]] = properties[["is_rent","is_sale","is_appraisal"]]. fillna('False')
    # fill '' in missing values of address
    properties[["address_1","address_2","address_3"]] = properties[["address_1","address_2","address_3"]]. fillna('')
    # concat all columns and include as a new column named categories
    def combined_features(row):
        return row['address_1']+" "+row['address_2']+" "+row['address_3']+" "+row['record_type']+" "+row['type']+" "+row['current_use']+" "+row['land_shape_type']+" "+row['site_position']+" "+str(row['is_rent'])+" "+str(row['is_sale'])+" "+str(row['is_appraisal'])
    properties["categories"] = properties.apply(combined_features, axis =1)
    # define a function for removing operators.
    def remove_operators(categories):
        return re.sub("[^a-zA-Z0-9 ]", "", categories)
    properties["categories"] = properties["categories"].apply(remove_operators)
    # count number ot terms, generate as matrix, and assign as a variable
    cv = CountVectorizer()
    matrix = cv.fit_transform(properties["categories"])
    #call the variable of matrix to calculate by Cosine Similarity
    properties_similar_scores = cosine_similarity(matrix)

##### Stage 3: Input Variable and Recommendation

In [63]:
#This is the input value of current_use that's input when users viewed.
current_use_user_viewed = "Farms"

def recommendation():
    # Input [type], Output [index] column.
    def get_index_from_current_use(current_use):
        return properties[properties.current_use == current_use]["index"].values[0]  
    property_index = get_index_from_current_use(current_use_user_viewed)
    # iterate similar scores, and assign as List
    list_similar_properties = list(enumerate(properties_similar_scores[property_index]))
    # Sort the list by descending
    desc_similar_properties = sorted(list_similar_properties, key=lambda x:x[1], reverse=True)
    # define function that return index
    def get_current_use_from_id(index):
        index = properties[properties.index == index]["index"].values[0]
        id = properties[properties.index == index]["property_id"].values[0]
        current_use = properties[properties.index == index]["current_use"].values[0]
        print(str(index) +", "+ str(id) +", "+ str(current_use)) # for deployment, this might be error because of data type. 
    # Recommend 10 similar properties back to users.
    i = 0
    for property in desc_similar_properties:
        print(get_current_use_from_id(property[0]))
        i = i + 1
        if i == 7:
            break